In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifake-vgg16-extracted-features/REAL_Test_Features.csv
/kaggle/input/cifake-vgg16-extracted-features/FAKE_Train_Features.csv
/kaggle/input/cifake-vgg16-extracted-features/FAKE_Test_Features.csv
/kaggle/input/cifake-vgg16-extracted-features/REAL_Train_Features.csv
/kaggle/input/demo-image-ship/ship.jpg
/kaggle/input/demo-vgg16-extracted/Fake_Demo_Features.csv
/kaggle/input/demo-vgg16-extracted/Real_Demo_Features.csv


In [28]:
!nvidia-smi

Thu May 18 18:04:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
!pip install "dask[complete]"
!pip install "dask-ml[complete]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.1/961.1 kB 17.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.3.2.1
    Uninstalling distributed-2023.3.2.1:
      Successfully uninstalled distributed-2023.3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
cuml 23.4.0 requires cupy-cuda11x<12.0.0a0,>=9.5.0, which is not installed.
raft-dask 23.4.0 requires dask==2023.3.2, but you have dask 2023.4.0 which is incompatible.
raft-dask 23.4.0 requires distributed==2023.3.2.1, but you have distributed 2023.4.0 which is incompatible.
dask-cudf 23.4.0 requires dask==2023.3.2, but you have dask 2023.4.0 which is incompatible.
dask-cudf 23.4.0 requires distributed==2023.3.2.1, but you have distributed 2023.4.0 which is incom

In [30]:
import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from time import time

In [31]:

df_real = pd.read_csv("/kaggle/input/vgg16-cifake/Real_Train_Features.csv")
df_fake = pd.read_csv("/kaggle/input/vgg16-cifake/Fake_Train_Features.csv")
df = pd.concat([df_real,df_fake],axis=0).drop('Unnamed: 0', axis=1)

df_sample = df.sample(frac=1.0, random_state=0)

X = df_sample.drop('class', axis=1)

le = LabelEncoder()
y = df_sample['class']
y = pd.DataFrame(le.fit_transform(y), columns=['class'])  # encode y to be numeric


In [ ]:
param_grid = {
    'learning_rate': [0.005,0.001,0.01, 0.1, 0.5],
    'max_depth':  np.arange(1,5 ,1),
    'n_estimators': np.arange(1500, 3100, 100)
}

In [ ]:
#model = KerasRegressor(build_fn=create_xgb_model, verbose=0)
model = xgb.XGBClassifier(
    tree_method='gpu_hist',
    return_train_score=True
)
gbtCV = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)


## Grid search on CPU

In [ ]:
with tf.device('/GPU:0'):
    time_start = time()
    gbtCV.fit(X, y)
    time_stop = time()
    
    print(f"Time elapsed: {(time_stop - time_start) / 60.0:.2f} minutes")

In [ ]:
gbtCV.best_score_

In [ ]:
gbtCV.best_params_

In [ ]:
ix = gbtCV.cv_results_['mean_test_score'].argmax()
#gbtCV.cv_results_['mean_train_score'][ix].round(4)
gbtCV.cv_results_['mean_test_score'][ix].round(4)


In [32]:
df_real = pd.read_csv("/kaggle/input/vgg16-cifake/REAL_Test_Features.csv")
df_fake = pd.read_csv("/kaggle/input/vgg16-cifake/Fake_Test_Features.csv")

df = pd.concat([df_real,df_fake],axis=0).drop('Unnamed: 0', axis=1)
X_test = df.drop('class', axis=1)

le = LabelEncoder()
y_test = df['class']
y_test = pd.DataFrame(le.fit_transform(y_test), columns=['class'])  # encode y to be numeric

## Test Score

In [ ]:
gbtCV.score(X_test,y_test)

In [27]:
import seaborn as sns

cv_results = gbtCV.cv_results_
df_results = pd.DataFrame(cv_results)
df_results = df_results[['param_max_depth', 'param_learning_rate', 'param_n_estimators', 'mean_test_score']]
df_results = df_results.pivot_table(index='param_n_estimators', columns=['param_max_depth', 'param_learning_rate'], values='mean_test_score')
sns.heatmap(df_results, cmap='viridis')

NameError: name 'gbtCV' is not defined

## Confusion Matrix

In [36]:
model = xgb.XGBClassifier(learning_rate= 0.5, max_depth= 4, n_estimators= 1920,tree_method='gpu_hist',
    return_train_score=True
)

In [37]:
model.fit(X,y)
y_p=model.predict(X_test)
prediction = le.inverse_transform(y_p)
prediction

[18:06:53] WARNING: ../src/learner.cc:767: 
Parameters: { "return_train_score" } are not used.



array(['Real', 'Real', 'Real', ..., 'Fake', 'Fake', 'Fake'], dtype=object)

In [38]:
confusion = pd.DataFrame()
confusion['target'] = y_test
confusion['prediction'] = prediction
confusion.head()

,target,prediction
0,Real,Real
1,Real,Real
2,Real,Real
3,Real,Real
4,Real,Real


In [39]:
# True Positive = Real Real
# False Positive = Fake Real
# True Negative = Fake Fake
# False Negative = Real Fake
confusion.value_counts()

target  prediction
Real    Real          9023
Fake    Fake          8995
        Real          1005
Real    Fake           977
dtype: int64